# Web Scrapping on Trendsmap - Top 1,000 Twitter Users

In [ ]:
import requests
from bs4 import BeautifulSoup
pages = range(10)
df = pd.DataFrame()
for page in pages:
    response = requests.get(f"https://www.trendsmap.com/twitter/top-users/{page}", headers={"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"})
    soup = BeautifulSoup(response.content, "html.parser")
    names = [a.text.strip() for a in soup.find_all("div", class_="inline-user-name")]
    screen_names = [a.text.strip() for a in soup.find_all("div", class_="inline-user-screen_name")]
    box_1 = pd.Series(names)
    box_2 = pd.Series(screen_names)
    df_page = pd.concat([box_1,box_2],axis=1,sort=False)
#     display(df_page)
    df = df.append(df_page,ignore_index=True)
    df.to_csv("../data/uid.csv",index=False)

### Get more information on each Twitter user_id

In [ ]:
import tweepy
import time
import csv
import pandas as pd
import json
from collections import OrderedDict
import datetime
import re

# 填写twitter提供的开发Key和secret
consumer_key = 'FFjagCHdNg0OdiOdLadsZvPws'
consumer_secret = 'igstCI7bZVvgBCEAymPodiJGSVuQ4zeBEDinw4x8BS41OCHKmW'
access_token = '4896380954-YUHiQ5CiYK2S1BIkFyI413tZrJ4c2K8EkfKcK7s'
access_token_secret = 'vfOWGqzmUSbmE0XO8iOxq4pTir5PKFIxJAqFfSeW2OIkl'

# 提交你的Key和secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# 获取类似于内容句柄的东西
api = tweepy.API(auth, proxy='127.0.0.1:7890')

In [ ]:
# 获取所有的user_id
with open('../data/uid.csv','r',encoding='utf-8') as f1:
    reader = csv.reader(f1)
    user_ids = [row[1] for row in reader]
    print(user_ids)
    print(len(user_ids))

for user_id in user_ids:
    print(f"正在查找用户ID=%s的用户信息..." % user_id)
    flag = 0
    while flag == 0:
        try:
            User = api.get_user(user_id=user_id)
        except Exception as e:
            print(repr(e))
            if repr(e) == "TweepError: [{'code': 50, 'message': 'User not found.'}]":
                print('用户不存在！')
                with open('user_not_found.csv','a',encoding='utf-8') as f:
                    writer = csv.writer(f)
                    writer.writerows(user_id)
                f.close()
                flag = 1
                break
            elif repr(e) == "TweepError: [{'code': 50, 'message': 'User not found.'}]":
                print('用户被冻结！')
                with open('user_has_been_suspended.csv','a',encoding='utf-8') as f:
                    writer = csv.writer(f)
                    writer.writerows(user_id)
                f.close()
                flag = 1
                break
            else:
                print("网络不稳定，正在尝试重新连接...")
                flag = 0
        else:
            flag = 1
            user_dict = {"user_id":user_id}
            user_dict["name"] = User.name
            user_dict["screen_name"] = User.screen_name
            user_dict["location"] = User.location
            user_dict["profile_location"] = User.profile_location
            user_dict["description"] = User.description
            user_dict["protected"] = User.protected
            user_dict["followers_count"] = User.followers_count
            user_dict["friends_count"] = User.friends_count
            user_dict["listed_count"] = User.listed_count
            user_dict["created_at"] = str(User.created_at)
            user_dict["favourites_count"] = User.favourites_count
            user_dict["utc_offset"] = User.utc_offset
            user_dict["time_zone"] = User.time_zone
            user_dict["geo_enabled"] = User.geo_enabled
            user_dict["verified"] = User.verified
            user_dict["statuses_count"] = User.statuses_count
            user_dict["lang"] = User.lang
            user_dict["contributors_enabled"] = User.contributors_enabled
            user_dict["is_translator"] = User.is_translator
            user_dict["is_translation_enabled"] = User.is_translation_enabled
            user_dict["profile_background_tile"] = User.profile_background_tile
            user_dict["profile_use_background_image"] = User.profile_use_background_image
            user_dict["has_extended_profile"] = User.has_extended_profile
            user_dict["default_profile"] = User.default_profile
            user_dict["default_profile_image"] = User.default_profile_image
            user_dict["following"] = User.following
            user_dict["follow_request_sent"] = User.follow_request_sent
            user_dict["notifications"] = User.notifications
            user_dict["translator_type"] = User.translator_type
            print(user_dict)
            json_str = json.dumps(user_dict, indent=4, ensure_ascii=False)
            with open(user_id + '.json', 'w',encoding='utf-8') as f2:
                f2.write(json_str)
            break
    # time.sleep(1)
    # f2.close()
f1.close()